Checking our data (one-hot). Make sure one_hot_csv_path is pointing to the file moves_one_hot.csv

In [2]:
import numpy as np
import pandas as pd

one_hot_csv_path = "/content/drive/My Drive/Colab Notebooks/moves_one_hot.csv"
one_hot_csv_df = pd.read_csv(one_hot_csv_path)
one_hot_csv_df.head()

,Name,UserRating,Grade,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,B15,B16,B17,B18,C1,...,I15,I16,I17,I18,J1,J2,J3,J4,J5,J6,J7,J8,J9,J10,J11,J12,J13,J14,J15,J16,J17,J18,K1,K2,K3,K4,K5,K6,K7,K8,K9,K10,K11,K12,K13,K14,K15,K16,K17,K18
0,+TAX,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,BELLA CIAO,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,PP1,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ZICK ZACK,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,WE ARE HERE TO TRAIN,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Basic analysis shows our data is skewed toward 6A+ (grade 2), since the imbalance is not too severe we leave it for now. Since there aren't too many samples in classes higher than 7C (grade 11), we assign them to grade 11 to indicate the category "7C or higher" To reduce noise, we only take problems with user rating 2 or higher (3 being highest), which leave us with 23861 samples to work with (reduced from 29205)

In [3]:
one_hot_csv_df = one_hot_csv_df[one_hot_csv_df['UserRating'] >= 2]
one_hot_csv_df.loc[one_hot_csv_df['Grade'] > 11, 'Grade'] = 11
one_hot_csv_df.head()

,Name,UserRating,Grade,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,B15,B16,B17,B18,C1,...,I15,I16,I17,I18,J1,J2,J3,J4,J5,J6,J7,J8,J9,J10,J11,J12,J13,J14,J15,J16,J17,J18,K1,K2,K3,K4,K5,K6,K7,K8,K9,K10,K11,K12,K13,K14,K15,K16,K17,K18
6,FISCHAA,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,SMZ12.1,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18,LOW HANGING MOON JUGS,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,SUNSHOT,3,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
22,MOVE BIC... EPS,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


We split data into training and testing set

In [0]:
from sklearn.model_selection import train_test_split
# Split data into train and test
random_state_seed = 42069
target_all = one_hot_csv_df.iloc[:,2].to_numpy()
feature_matrix_all = one_hot_csv_df.iloc[:,3:].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(feature_matrix_all, target_all, train_size=0.75, random_state=random_state_seed)

Let n be the number of unique classes. To make use of the fact that our target data is ordinal, instead of directly predicting the class label using one multiclass classifier, we train n-1 binary classifiers, where each classifier is responsible for producing the probability of a sample having a grade greater than i, where 0<=i<=n-1. In the end, we can predict the probability of the sample belonging to class y by calculating the difference between the probability of it having a grade greater than y-1 and the probability of it having a grade greater than y. For example, prob(class = 0) = 1 - prob(class>0), prob(class = 4) = prob(class > 3) - prob(class > 4), prob(class = n) = prob(class > n-1)

In [0]:
# Code reference: https://towardsdatascience.com/simple-trick-to-train-an-ordinal-regression-with-any-classifier-6911183d2a3c
from sklearn.base import clone
#from copy import deepcopy
class OrdinalClassifier():

  def __init__(self, model):
    self.model = model
    self.models = {}
    
  def fit(self, X, y):
    for i in range(11):
      # for each k - 1 ordinal value we fit a binary classification problem
      binary_y = (y > i).astype(np.uint8)
      model = clone(self.model)
      model.fit(X, binary_y)
      self.models[i] = model

  # Returns a matrix of size 11 x N where N = number of samples  
  def predict_proba(self, X):
    # models_predict is a dict of prob(y > i) : Nx2 matrix of prob(y<=i),prob(y>i) 
    models_predict = {i:self.models[i].predict_proba(X) for i in range(len(self.models))}
    predicted = []
    for i in range(12):
        if i == 0:
          # V1 = 1 - Pr(y > V1)
          predicted.append(1 - models_predict[i][:,1])
        elif i != 11:
          # Vi = Pr(y > Vi-1) - Pr(y > Vi)
          predicted.append(models_predict[i-1][:,1] - models_predict[i][:,1])
        else:
          # Vk = Pr(y > Vk-1)
          predicted.append(models_predict[i-1][:,1])
    return np.vstack(predicted).T
    
  def predict(self, X):
    return np.argmax(self.predict_proba(X), axis=1)

Experiment with training with/without the ordinal information. Starting with Naive Bayes. We also define the evaluation metric n-off accuracy, where we consider a prediction to be correct as long as the difference between the actual difficulty and the prediction is within n. For example, (prediction:2, actual: 4) will be considered as a correct prediction in 2-off accuracy but incorrect in 1-off accuracy. We do this to take into account that due to the subjective nature of climbing grades, climbers generally tolerate difficulty level that are 1 level off of what they would expect it to be.

In [6]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

# Prediction is considered correct as long as it is within +- n
def n_off_accuracy(test,pred,n):
  difference = np.absolute(test - pred)
  return (difference <= n).sum() / len(test)

naive_bayes_model = BernoulliNB()
naive_bayes_model.fit(X_train, y_train)
naive_bayes_model_prediction = naive_bayes_model.predict(X_test)
print("Naive Bayes Exact Accuracy: Without Ordinal Classifier")
print(accuracy_score(y_test, naive_bayes_model_prediction))
print("Naive Bayes 1-off Accuracy: Without Ordinal Classifier")
print(n_off_accuracy(y_test,naive_bayes_model_prediction,1))
print("Naive Bayes 2-off Accuracy: Without Ordinal Classifier")
print(n_off_accuracy(y_test,naive_bayes_model_prediction,2))
print("Naive Bayes Test MAE: Without Ordinal Classifier")
print(mean_absolute_error(y_test,naive_bayes_model_prediction))

print()

naive_bayes_model_ordinal = OrdinalClassifier(BernoulliNB())
naive_bayes_model_ordinal.fit(X_train, y_train)
naive_bayes_ordinal_prediction = naive_bayes_model_ordinal.predict(X_test)
print("Naive Bayes Exact Accuracy: With Ordinal Classifier")
print(accuracy_score(y_test, naive_bayes_ordinal_prediction))
print("Naive Bayes 1-off Accuracy: With Ordinal Classifier")
print(n_off_accuracy(y_test,naive_bayes_ordinal_prediction,1))
print("Naive Bayes 2-off Accuracy: With Ordinal Classifier")
print(n_off_accuracy(y_test,naive_bayes_ordinal_prediction,2))
print("Naive Bayes Test MAE: With Ordinal Classifier")
print(mean_absolute_error(y_test,naive_bayes_ordinal_prediction))

Naive Bayes Exact Accuracy: Without Ordinal Classifier
0.3169627891384512
Naive Bayes 1-off Accuracy: Without Ordinal Classifier
0.5608447871270533
Naive Bayes 2-off Accuracy: Without Ordinal Classifier
0.7696949379818974
Naive Bayes Test MAE: Without Ordinal Classifier
1.5980556486758297

Naive Bayes Exact Accuracy: With Ordinal Classifier
0.2666778410995642
Naive Bayes 1-off Accuracy: With Ordinal Classifier
0.4772041568890379
Naive Bayes 2-off Accuracy: With Ordinal Classifier
0.7252765672142139
Naive Bayes Test MAE: With Ordinal Classifier
1.8749580958766343


Using ordinal information does not seem to have added benefits while also increases the training time, we therefore discard this strategy in other models. Next up we attempt using random forest and evaluate it using similar metric. Training takes about 5-10 seconds.

In [31]:
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier(random_state=random_state_seed)
# Training takes about 5-10 seconds
random_forest_model.fit(X_train, y_train)
random_forest_model_prediction = random_forest_model.predict(X_test)

print("Random Forest Exact Accuracy:")
print(accuracy_score(y_test, random_forest_model_prediction))
print("Random Forest 1-off Accuracy:")
print(n_off_accuracy(y_test,random_forest_model_prediction,1))
print("Random Forest 2-off Accuracy:")
print(n_off_accuracy(y_test,random_forest_model_prediction,2))
print("Random Forest Test MAE:")
print(mean_absolute_error(y_test, random_forest_model_prediction))

Random Forest Exact Accuracy:
0.33791485082132083
Random Forest 1-off Accuracy:
0.58900435802883
Random Forest 2-off Accuracy:
0.7955078779751927
Random Forest Test MAE:
1.4644653033858532


Next up, SVM. Worth noting that our accuracy here is slightly better than the best accuracy in previous work (36.5%) http://cs229.stanford.edu/proj2017/final-reports/5232206.pdf . Training takes about 2-3 minutes

In [32]:
from sklearn.svm import SVC

svm_model = SVC(random_state=random_state_seed)
svm_model.fit(X_train, y_train)
# Training takes about 2-3 minutes
svm_model_prediction = svm_model.predict(X_test)
print("SVM Test Accuracy:")
print(accuracy_score(y_test, svm_model_prediction))
print("SVM 1-off Accuracy:")
print(n_off_accuracy(y_test,svm_model_prediction,1))
print("SVM 2-off Accuracy:")
print(n_off_accuracy(y_test,svm_model_prediction,2))
print("SVM Test MAE:")
print(mean_absolute_error(y_test, svm_model_prediction))

SVM Test Accuracy:
0.3672477371773383
SVM 1-off Accuracy:
0.6101240362051625
SVM 2-off Accuracy:
0.8183037210861549
SVM Test MAE:
1.3461280590010056


Next up, a simple feedforward neural network defined using code from HW4

In [0]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable

torch.manual_seed(random_state_seed)

class HwNet(torch.nn.Module):
  def __init__(self, num_layers, units ,final_activation):
    super(HwNet, self).__init__()
    self.num_layers = num_layers
    self.final_activation = final_activation
    self.units = units
    layers = []
    for i in range(len(units)-1):
      layers.append(torch.nn.Linear(units[i],units[i+1]))
    self.layers = torch.nn.ModuleList(layers)

  def forward(self, x):
    for i in range(len(self.layers)):
      if i == len(self.layers)-1:
        if (self.final_activation == "sigmoid"):
          x = torch.sigmoid(self.layers[i](x))
        elif (self.final_activation == "tanh"):
          x = F.tanh(self.layers[i](x))
        elif (self.final_activation == "relu"):
          x = F.relu(self.layers[i](x))
        elif (self.final_activation == "identity"):
          x = self.layers[i](x)
      else:
        x = F.relu(self.layers[i](x))
    return x

  def print_info(self):
    print("Weights and Biases")
    for param in self.parameters():
      print(param.data)
    print("Final Activation")
    print(self.final_activation)

  def train(self,train_data,train_labels):
    self.model_reset()
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(self.parameters(), lr=0.1)
    for t in range(500):
      # Forward pass: Compute predicted y by passing x to the model
      y_pred = self.forward(train_data)

      # Compute and print loss
      loss = criterion(y_pred, train_labels)
      # if t % 100 == 99:
      #     print(t, loss.item())

      # Zero gradients, perform a backward pass, and update the weights.
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    # print("Training finished")

  def predict(self, x_val):
    x = Variable(x_val, requires_grad=False)
    output = self.forward(x)
    output_numpy = np.argmax(output.data.numpy(), axis=1)
    return output_numpy

  def model_reset(self):
    for layer in self.layers:
      layer.reset_parameters()


Some additional data processing and helper function before feeding into the NN

In [0]:
trainX = torch.from_numpy(X_train).float()
testX = torch.from_numpy(X_test).float()
trainY = torch.from_numpy(y_train)
testY = torch.from_numpy(y_test)

def evaluate_model(model,trainX,trainY,testX,testY):
  model.train(trainX,trainY)
  model_prediction_test = model.predict(testX)
  model_test_accuracy = accuracy_score(model_prediction_test,testY.data.numpy())
  model_1_off_accuracy = n_off_accuracy(model_prediction_test,testY.data.numpy(),1)
  model_2_off_accuracy = n_off_accuracy(model_prediction_test,testY.data.numpy(),2)
  model_mae = mean_absolute_error(model_prediction_test, testY.data.numpy())
  return model_test_accuracy,model_1_off_accuracy,model_2_off_accuracy,model_mae

Train and evaluate the neural network. This takes about 5-6 minutes.

In [9]:
import time
# Experiment with different neurons in different layers
# There are 11*18 = 198 features
# Training takes about 5-6 minutes
model_2_512_sig = HwNet(4, [198,512,512,12],'sigmoid')
model_test_accuracy,model_1_off_accuracy,model_2_off_accuracy,model_mae = evaluate_model(model_2_512_sig,trainX,trainY,testX,testY)
print("Linear NN 198-512-512-12 Accuracy:")
print(model_test_accuracy)
print("Linear NN 198-512-512-12 1-off Accuracy:")
print(model_1_off_accuracy)
print("Linear NN 198-512-512-12 2-off Accuracy:")
print(model_2_off_accuracy)
print("Linear NN 198-512-512-12 Test MAE:")
print(model_mae)

Linear NN 198-512-512-12 Accuracy:
0.26952732148843445
Linear NN 198-512-512-12 1-off Accuracy:
0.3984244049614482
Linear NN 198-512-512-12 2-off Accuracy:
0.5636942675159236
Linear NN 198-512-512-12 Test MAE:
2.7346630908481395


The result of using one hot vector is not too satisfactory, let's see if that changes when we treat the problem as a natural language problem. Make sure moves_reordered_compact_path points to the file moves_reordered_path.csv

In [13]:
# Checking our data and filtering it the same way we did before
moves_reordered_compact_path = "/content/drive/My Drive/Colab Notebooks/moves_reordered_compact.csv"
moves_reordered_df = pd.read_csv(moves_reordered_compact_path)
moves_reordered_df = moves_reordered_df[moves_reordered_df['UserRating'] >= 2]
moves_reordered_df.loc[moves_reordered_df['Grade'] > 11, 'Grade'] = 11
moves_reordered_df.head()

,Name,UserRating,Holds,Grade
6,FISCHAA,2,H5 E3 F8 I7 J11 D10 E13 A14 C16 B18 E18,2
7,SMZ12.1,2,D5 F5 F8 I8 I7 H10 F12 D12 E13 D15 C16 D9 F18 B18,2
18,LOW HANGING MOON JUGS,2,J1 F5 I8 F11 F12 D15 H15 F18,2
19,SUNSHOT,3,G6 K5 G8 D9 C13 F13 D15 B18,5
22,MOVE BIC... EPS,3,F5 H5 I8 H10 G13 E13 H16 F18,2


To make use of pytorch nlp capabilities, we convert the data to json line format for easier processing. Running this code wll generate train.json and test.json. If train.json and test.json already exist there is no need to run this block of code

In [0]:
import json

feature_matrix_all = moves_reordered_df.loc[:, ['Holds']].to_numpy()
target_all = moves_reordered_df.loc[:, ['Grade']].to_numpy()
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(feature_matrix_all, target_all, train_size=0.75, random_state=random_state_seed)

with open('train.json', 'w') as outfile:
  for pair in zip(X_train_2,y_train_2):
    row = {}
    holds = pair[0].item(0).split(' ')
    grade = pair[1].item(0)
    row['Holds'] = holds
    row['Grade'] = grade
    json.dump(row, outfile)
    outfile.write('\n')

with open('test.json', 'w') as outfile:
  for pair in zip(X_test_2,y_test_2):
    row = {}
    holds = pair[0].item(0).split(' ')
    grade = pair[1].item(0)
    row['Holds'] = holds
    row['Grade'] = grade
    json.dump(row, outfile)
    outfile.write('\n')

We first load up the data in a format that is compatible with torchtext and split them into train, validation, and test. We also set up some miscellaneous settings that will help us train the neural network. Make sure the train_test_data_directory is pointing to the directory containing train.json and test.json

In [0]:
import torch
import random
from torchtext import data
from torchtext import datasets

BATCH_SIZE = 64

train_test_data_directory = '/content/drive/My Drive/Colab Notebooks'
HOLDS = data.Field()
GRADE = data.LabelField()
fields = {'Holds': ('holds', HOLDS), 'Grade': ('grade', GRADE)}
train_data, test_data = data.TabularDataset.splits(
                            path = train_test_data_directory,
                            train = 'train.json',
                            test = 'test.json',
                            format = 'json',
                            fields = fields
)
train_data, valid_data = train_data.split(random_state = random.seed(random_state_seed))

HOLDS.build_vocab(train_data)
GRADE.build_vocab(train_data)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort=False)

We define a CNN model that can be used on sequences.
Code Rederence:https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/5%20-%20Multi-class%20Sentiment%20Analysis.ipynb

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
               dropout, pad_idx):
    super().__init__()     
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.convs = nn.ModuleList([
                                nn.Conv2d(in_channels = 1, 
                                          out_channels = n_filters, 
                                          kernel_size = (fs, embedding_dim)) 
                                for fs in filter_sizes
                                ])
        
    self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)   
    self.dropout = nn.Dropout(dropout)
        
  def forward(self, text):
    #text = [sent len, batch size]        
    text = text.permute(1, 0)                
    #text = [batch size, sent len]       
    embedded = self.embedding(text)               
    #embedded = [batch size, sent len, emb dim]      
    embedded = embedded.unsqueeze(1)      
    #embedded = [batch size, 1, sent len, emb dim]
    conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
    #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
    pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
    #pooled_n = [batch size, n_filters]
    cat = self.dropout(torch.cat(pooled, dim = 1))
    #cat = [batch size, n_filters * len(filter_sizes)]
    return self.fc(cat)

Defining helper functions

In [0]:
import torch.optim as optim
import time

def categorical_accuracy(preds, y):
  max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
  correct = max_preds.squeeze(1).eq(y)
  return correct.sum() / torch.FloatTensor([y.shape[0]])

def train(model, iterator, criterion, parameters):
  optimizer = optim.Adam(model.parameters(),lr=parameters['lr'])
  #optimizer = optim.SGD(model.parameters(),lr=parameters['lr'])
  criterion = nn.CrossEntropyLoss()
  epoch_loss = 0
  epoch_acc = 0
  model.train()
  for batch in iterator:
    optimizer.zero_grad()
    predictions = model(batch.holds)
    loss = criterion(predictions, batch.grade)
    acc = categorical_accuracy(predictions, batch.grade)
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()
    epoch_acc += acc.item()
  return epoch_loss / len(iterator), epoch_acc / len(iterator), model

def evaluate(model, iterator, criterion):
  epoch_loss = 0
  epoch_acc = 0
  epoch_1_off_acc = 0
  epoch_2_off_acc = 0
  epoch_mae = 0
  model.eval()
  with torch.no_grad():
    for batch in iterator:
      predictions = model(batch.holds)
      loss = criterion(predictions, batch.grade)
      acc = categorical_accuracy(predictions, batch.grade)
      epoch_1_off_acc += n_off_accuracy(np.argmax(predictions.data.numpy(), axis=1),batch.grade.data.numpy(),1)
      epoch_2_off_acc += n_off_accuracy(np.argmax(predictions.data.numpy(), axis=1),batch.grade.data.numpy(),2)
      epoch_mae += mean_absolute_error(np.argmax(predictions.data.numpy(), axis=1), batch.grade.data.numpy())
      epoch_loss += loss.item()
      epoch_acc += acc.item()        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_1_off_acc / len(iterator), epoch_2_off_acc / len(iterator),epoch_mae / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Install Bayesian Optimization dependency

In [0]:
!pip3 install ax-platform

Define model parameters

In [0]:
from ax.service.managed_loop import optimize

INPUT_DIM = len(HOLDS.vocab)
# EMBEDDING_DIM = 10
# N_FILTERS = 100
# DROPOUT = 0.5
# LR = 0.1 on adam
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(GRADE.vocab)
PAD_IDX = HOLDS.vocab.stoi[HOLDS.pad_token]

def train_evaluate(parameterization):
  criterion = nn.CrossEntropyLoss()
  criterion = criterion.to(device)
  model = CNN(INPUT_DIM, round(parameterization['embedding']), round(parameterization['filters']), FILTER_SIZES, OUTPUT_DIM, parameterization['dropout'], PAD_IDX)
  model = model.to(device)
  _,_,net = train(model=model, iterator=train_iterator,criterion=criterion, parameters=parameterization)
  valid_loss,_,_,_,_ = evaluate(model=net,iterator=valid_iterator,criterion=criterion)
  return valid_loss

Finding the best hyperparameters for the model and saves the result model. Takes about 1-2 minutes

In [155]:
best_parameters, values, _, _ = optimize(
    parameters=[
        {"name": "lr", "type": "range", "bounds": [1e-6, 0.4], "log_scale": True},
        {"name": "dropout", "type": "range", "bounds": [0.2, 0.6]},
        {"name": "embedding", "type": "range", "bounds": [1,300]},
        {"name": "filters", "type": "range", "bounds":[1,200]}
    ],
    evaluation_function=train_evaluate,
    minimize=True
)

print(best_parameters)
print(values)

[INFO 04-24 00:56:37] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 5 arms, GPEI for subsequent arms], generated 0 arm(s) so far). Iterations after 5 will take longer to generate due to model-fitting.
[INFO 04-24 00:56:37] ax.service.managed_loop: Started full optimization with 20 steps.
[INFO 04-24 00:56:37] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-24 00:56:45] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-24 00:56:52] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-24 00:56:56] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-24 00:57:00] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-24 00:57:22] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-24 00:57:35] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-24 00:57:42] ax.service.managed_loop: Running optimization t

{'lr': 0.0011613296155845206, 'dropout': 0.4068616587758692, 'embedding': 117, 'filters': 168}
({'objective': 1.82704477985812}, {'objective': {'objective': 2.458726097556533e-09}})


Train model with these parameters and save the model that does best on valdiation set from 5 epochs.

In [177]:
from decimal import Decimal

best_lr = best_parameters['lr']
best_dropout = best_parameters['dropout']
best_embedding_dimension = best_parameters['embedding']
best_filter_nums = best_parameters['filters']
best_lr_decimal = round(Decimal(best_lr),4)
best_dropout_decimal = round(Decimal(best_dropout),4)

best_model = CNN(INPUT_DIM, best_embedding_dimension, best_filter_nums, FILTER_SIZES, OUTPUT_DIM, best_dropout, PAD_IDX)
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc, _ = train(best_model, train_iterator,criterion,best_parameters)
    valid_loss, valid_acc,model_1_off_accuracy,model_2_off_accuracy,model_mae = evaluate(best_model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(best_model.state_dict(), f'loss-{best_valid_loss:.3f}-eb-{best_embedding_dimension}-ft-{best_filter_nums}-lr-{best_lr_decimal}-do-{best_dropout_decimal}.pt')
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 13s
	Train Loss: 2.010 | Train Acc: 30.01%
	 Val. Loss: 1.848 |  Val. Acc: 33.22%
Epoch: 02 | Epoch Time: 0m 13s
	Train Loss: 1.815 | Train Acc: 35.19%
	 Val. Loss: 1.818 |  Val. Acc: 35.01%
Epoch: 03 | Epoch Time: 0m 13s
	Train Loss: 1.718 | Train Acc: 38.11%
	 Val. Loss: 1.857 |  Val. Acc: 34.22%
Epoch: 04 | Epoch Time: 0m 13s
	Train Loss: 1.640 | Train Acc: 41.35%
	 Val. Loss: 1.860 |  Val. Acc: 34.63%
Epoch: 05 | Epoch Time: 0m 13s
	Train Loss: 1.558 | Train Acc: 43.98%
	 Val. Loss: 1.868 |  Val. Acc: 32.76%


Test the model on test set

In [178]:
best_model = CNN(INPUT_DIM, best_embedding_dimension, best_filter_nums, FILTER_SIZES, OUTPUT_DIM, best_dropout, PAD_IDX)
best_model.load_state_dict(torch.load(f'loss-{best_valid_loss:.3f}-eb-{best_embedding_dimension}-ft-{best_filter_nums}-lr-{best_lr_decimal}-do-{best_dropout_decimal}.pt'))
test_loss, test_acc, model_1_off_accuracy,model_2_off_accuracy, model_mae = evaluate(best_model, test_iterator, criterion)

print("CNN Exact Accuracy:")
print(test_acc)
print("CNN 1-off Accuracy:")
print(model_1_off_accuracy)
print("CNN 2-off Accuracy:")
print(model_2_off_accuracy)
print("CNN Test MAE:")
print(model_mae)

#print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

CNN Exact Accuracy:
0.3459821429024351
CNN 1-off Accuracy:
0.5635210866261399
CNN 2-off Accuracy:
0.7385068389057751
CNN Test MAE:
1.8363174392097263


As a baseline comparison, we create a FastText model used in "Bag of Tricks for Efficient Text Classification" https://arxiv.org/abs/1607.01759 

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
  def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):  
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
    self.fc = nn.Linear(embedding_dim, output_dim)

  def forward(self, text):
    #text = [sent len, batch size]
    embedded = self.embedding(text)
    #embedded = [sent len, batch size, emb dim]
    embedded = embedded.permute(1, 0, 2)
    #embedded = [batch size, sent len, emb dim]
    pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
    #pooled = [batch size, embedding_dim]
    return self.fc(pooled)

model = FastText(INPUT_DIM, best_embedding_dimension, OUTPUT_DIM, PAD_IDX)

Training FastText

In [182]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc, _ = train(model, train_iterator, criterion, best_parameters)
    vvalid_loss, valid_acc,model_1_off_accuracy,model_2_off_accuracy,model_mae = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'fast-text-model.pt')
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 2.144 | Train Acc: 27.90%
	 Val. Loss: 1.868 |  Val. Acc: 29.85%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 1.935 | Train Acc: 32.03%
	 Val. Loss: 1.868 |  Val. Acc: 31.88%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 1.877 | Train Acc: 33.53%
	 Val. Loss: 1.868 |  Val. Acc: 32.87%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 1.847 | Train Acc: 33.96%
	 Val. Loss: 1.868 |  Val. Acc: 32.95%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 1.828 | Train Acc: 34.64%
	 Val. Loss: 1.868 |  Val. Acc: 33.28%


Finally test fasttext on test set

In [183]:
model.load_state_dict(torch.load('fast-text-model.pt'))
test_loss, test_acc, model_1_off_accuracy,model_2_off_accuracy, model_mae = evaluate(model, test_iterator, criterion)
print("FastText Exact Accuracy:")
print(test_acc)
print("FastText 1-off Accuracy:")
print(model_1_off_accuracy)
print("FastText 2-off Accuracy:")
print(model_2_off_accuracy)
print("FastText Test MAE:")
print(model_mae)
#print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

FastText Exact Accuracy:
0.2959014058430144
FastText 1-off Accuracy:
0.4549297112462006
FastText 2-off Accuracy:
0.6078077507598784
FastText Test MAE:
2.516337386018237


To test the statistical significance of our result, we compare them against a dummy model that generates predictions using the same distribution as the training set’s class distribution and a SVM model that only uses the number of holds in a problem as the feature

In [25]:
X_train_num_moves = X_train.sum(axis=1).reshape(-1, 1)
X_test_num_moves = X_test.sum(axis=1).reshape(-1, 1)

from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC

dummy_clf = DummyClassifier(strategy="stratified",random_state=random_state_seed,constant=2)
dummy_clf.fit(X_train_num_moves, y_train)
dummy_clf_predictions = dummy_clf.predict(X_test_num_moves)
print("Dummy Exact Accuracy (stratified):")
print(accuracy_score(y_test, dummy_clf_predictions))
print("Dummy 1-off Accuracy (stratified):")
print(n_off_accuracy(y_test,dummy_clf_predictions,1))
print("Dummy 2-off Accuracy (stratified):")
print(n_off_accuracy(y_test,dummy_clf_predictions,2))
print("Dummy Test MAE (stratified):")
print(mean_absolute_error(y_test, dummy_clf_predictions))

print()

svm_model = SVC(random_state=random_state_seed)
svm_model.fit(X_train_num_moves, y_train)
# Training takes about 10-20 seconds
svm_model_prediction = svm_model.predict(X_test_num_moves)
print("SVM Eaxct Accuracy (using number of moves only):")
print(accuracy_score(y_test, svm_model_prediction))
print("SVM 1-off Accuracy (using number of moves only):")
print(n_off_accuracy(y_test,svm_model_prediction,1))
print("SVM 2-off Accuracy (using number of moves only):")
print(n_off_accuracy(y_test,svm_model_prediction,2))
print("SVM Test MAE (using number of moves only):")
print(mean_absolute_error(y_test, svm_model_prediction))

Dummy Exact Accuracy (stratified):
0.13476366074421722
Dummy 1-off Accuracy (stratified):
0.30958766342608113
Dummy 2-off Accuracy (stratified):
0.4927924907810929
Dummy Test MAE (stratified):
3.0424069728461283

SVM Eaxct Accuracy (using number of moves only):
0.2696949379818974
SVM 1-off Accuracy (using number of moves only):
0.4054642976868924
SVM 2-off Accuracy (using number of moves only):
0.5714046262152196
SVM Test MAE (using number of moves only):
2.659068052296346
